#### Домашнее задание 11 november

    - выбрать item-based/user-based подход для работы с датасетом
    - реализовать модель подсчета рейтинга с использованием кластеризации (количество кластеров обосновать)
    - посчитать рейтинги с помощью матричной факторизации SVD/NMF
    - сравнить три способа (имеющийся с корреляцией Пирсона, кластеризация, матричная факторизация)

In [53]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import surprise
%matplotlib inline

In [54]:
rmse = lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))

In [55]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv', parse_dates=['timestamp'])
ratings = ratings.sample(frac=1).reset_index(drop=True)

In [56]:
ratings.head()

,userId,movieId,rating,timestamp
0,599,161008,2.5,1519100249
1,74,3949,5.0,1207501199
2,605,2870,3.5,1277097222
3,282,296,4.0,1378490071
4,121,432,2.0,847656100


In [57]:
RATIO = 0.2

In [58]:
test = ratings[-int(ratings.shape[0]*RATIO):]
test = test.drop(["timestamp"], axis =1)
ratings.loc[test.index,"rating"] = 0
train = ratings.drop(["timestamp"], axis =1)


In [59]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator

In [60]:
train_set, test_set = train.copy(), test.copy()
lower_rating, upper_rating = train_set["rating"].min(), train_set["rating"].max()
data = surprise.Dataset.load_from_df(train_set, surprise.Reader(rating_scale=(lower_rating,upper_rating)))
trainset = data.build_full_trainset()
sim_options = {
    'name': 'MSD',
    'user_based': True
}
knn = surprise.KNNBasic(k=100, sim_options = sim_options)
knn.fit(trainset)
knn_predict = lambda row: knn.predict(uid=row.userId,iid=row.movieId).est
test_set["predict"] = test_set.apply(knn_predict, axis = 1)
rmse_result = rmse(test_set["predict"].tolist(), test_set["rating"].tolist())


Computing the msd similarity matrix...
Done computing similarity matrix.


In [61]:
rmse_result

1.8937340334639678

In [62]:
test_set

,userId,movieId,rating,predict
80669,404,224,3.0,2.462487
80670,567,136020,1.0,1.810331
80671,40,153,3.0,2.027371
80672,238,1092,4.0,2.107647
80673,211,1221,3.5,2.931317
...,...,...,...,...
100831,131,356,3.0,1.720171
100832,593,4973,3.5,2.825200
100833,525,32587,3.5,2.783945
100834,469,2993,3.0,2.179552


## Clusterization (user based)

In [63]:
train_set, test_ser = train.copy(), test.copy() 

In [64]:
cluster_df = pd.pivot_table(train_set, values='rating', index='userId',
                                           columns='movieId', fill_value=0).T

In [65]:
cluster_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,4.0,2.5,4.0,2.5,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn import  metrics

In [67]:
COUNT_CLUSTERS = 120

In [68]:
model = KMeans(COUNT_CLUSTERS,n_jobs=-1)
model.fit(cluster_df)

KMeans(n_clusters=120, n_jobs=-1)

In [69]:
len(model.labels_)

9724

In [70]:
cluster_df["cluster"] = model.labels_

In [71]:
cluster_df

userId,1,2,3,4,5,6,7,8,9,10,...,602,603,604,605,606,607,608,609,610,cluster
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,4.0,2.5,4.0,2.5,0.0,0.0,52
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,4.0,0.0,5.0,3.5,0.0,0.0,0.0,0.0,0.0,15
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34


In [72]:
cluster_df = cluster_df\
            .groupby("cluster")\
            .transform(lambda x: x.replace(0,x.mean()))

In [73]:
cluster_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,3.000000,4.000000,2.500000,4.000000,2.500000,0.000000,0.000000
2,1.176471,0.132353,0.014706,0.882353,0.000000,4.000000,0.191176,4.00000,0.117647,0.000000,...,0.500000,4.000000,1.382353,5.000000,3.500000,1.176471,0.735294,2.161765,0.000000,0.955882
3,4.000000,0.045977,0.005747,0.206897,0.149425,5.000000,0.000000,0.16092,0.034483,0.000000,...,0.000000,0.850575,0.344828,0.954023,0.034483,0.568966,0.057471,0.660920,0.034483,0.086207
4,0.009901,0.001597,0.015490,0.010540,0.000000,0.035452,0.002395,0.00000,0.004152,0.012137,...,0.007027,0.002874,0.009582,0.006068,0.025551,0.002555,0.007985,0.017247,0.002874,0.003833
5,0.218391,0.045977,0.005747,0.206897,0.149425,5.000000,0.000000,0.16092,0.034483,0.000000,...,0.000000,0.850575,0.344828,3.000000,0.034483,0.568966,0.057471,0.660920,0.034483,0.086207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.009901,0.001597,0.015490,0.010540,0.000000,0.035452,0.002395,0.00000,0.004152,0.012137,...,0.007027,0.002874,0.009582,0.006068,0.025551,0.002555,0.007985,0.017247,0.002874,0.003833
193583,0.009901,0.001597,0.015490,0.010540,0.000000,0.035452,0.002395,0.00000,0.004152,0.012137,...,0.007027,0.002874,0.009582,0.006068,0.025551,0.002555,0.007985,0.017247,0.002874,0.003833
193585,0.009901,0.001597,0.015490,0.010540,0.000000,0.035452,0.002395,0.00000,0.004152,0.012137,...,0.007027,0.002874,0.009582,0.006068,0.025551,0.002555,0.007985,0.017247,0.002874,0.003833


In [74]:
test_set

,userId,movieId,rating,predict
80669,404,224,3.0,2.462487
80670,567,136020,1.0,1.810331
80671,40,153,3.0,2.027371
80672,238,1092,4.0,2.107647
80673,211,1221,3.5,2.931317
...,...,...,...,...
100831,131,356,3.0,1.720171
100832,593,4973,3.5,2.825200
100833,525,32587,3.5,2.783945
100834,469,2993,3.0,2.179552


In [75]:
test_set["predict"] = test_set.apply(lambda row: cluster_df.loc[row.movieId, row.userId], axis = 1)

In [76]:
rmse_result = rmse(test_set["predict"].tolist(), test_set["rating"].tolist())

In [77]:
rmse_result

3.182059402023996

In [78]:
test_set

,userId,movieId,rating,predict
80669,404,224,3.0,0.494253
80670,567,136020,1.0,0.800000
80671,40,153,3.0,1.785714
80672,238,1092,4.0,0.056338
80673,211,1221,3.5,2.250000
...,...,...,...,...
100831,131,356,3.0,0.000000
100832,593,4973,3.5,0.000000
100833,525,32587,3.5,1.870370
100834,469,2993,3.0,1.133803


## SVD approach

In [79]:
train_set = train.copy()
test_set = test.copy()

In [80]:
lower_rating = train_set["rating"].min()
upper_rating = train_set["rating"].max()
data = surprise.Dataset.load_from_df(train_set, surprise.Reader(rating_scale=(lower_rating,upper_rating)))
trainset = data.build_full_trainset()

In [81]:
algorithm_svd = surprise.SVDpp(n_factors=2, n_epochs=20, lr_all = 0.01, reg_all=0.1 )
algorithm_svd.fit(trainset)
test_set["predict"] = test_set.apply(lambda row: algorithm_svd.predict(uid=row.userId,iid=row.movieId).est, axis = 1)
rmse_result = rmse(test_set["predict"].tolist(), test_set["rating"].tolist())


In [82]:
rmse_result

2.3238450815362715

In [83]:
test_set

,userId,movieId,rating,predict
80669,404,224,3.0,2.446784
80670,567,136020,1.0,0.749694
80671,40,153,3.0,2.003661
80672,238,1092,4.0,2.344163
80673,211,1221,3.5,3.461785
...,...,...,...,...
100831,131,356,3.0,2.663433
100832,593,4973,3.5,2.881031
100833,525,32587,3.5,1.577133
100834,469,2993,3.0,1.303716


## Итоги

### Пирсон: 
лучшая точность, но совершенно не масштабируемо, из-за n^2 сложности ( в идеале, по факту n*100). На большом объеме данных способ будет работать плохо.

### SVD:
Матричная факторизация способна масштабироваться, но требует большого количества вычислений из-за градиентного поиска. Точность и стабильность, возможность выделять скрытые свойства данных. Но это при условии наличия мощной машины, т.к. количество операций - большое, причем один градиентный шаг имеет сложность (n+m)*d. 

### Кластеризация:
Для данной задачи отработала хуже всего, из-за разреженности данных. Не стоит ее использовать для данной задачи, хоть ошибка в принципе и упала.
